# Subways and Social Distancing – Data Collection

I started looking into subway usage in New York City shortly after a [stay-at-home](https://www.nytimes.com/article/what-is-shelter-in-place-coronavirus.html) order was issued on March 24, 2020. For me personally as a graduate student, it felt like serious social distancing measures began around two weeks prior to that when university classes were moved to remote instruction, and I was hoping to get a better sense of how others in New York have been using or not using public transportation.

I found this notebook while looking through my files from the time, and I wanted to update it with the most recent data, including subway usage after vaccination milestones and the latest surge related to the Omicron variant.

**Last updated: January 22, 2022** 

In [1]:
import requests
from bs4 import BeautifulSoup

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

from io import StringIO
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.dates as mdates
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU

import seaborn as sns
import plotly.express as px

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import time
import datetime as dt
from datetime import datetime

%matplotlib inline

### Collecting data

To start, I collected data from the New York Metropolitan Transportation Authority's weekly subway [turnstile database](http://web.mta.info/developers/turnstile.html). I wanted to look at all data from 2020 onward and compare to the pre-pandemic days of 2019.

In [2]:
subway_turnstile_url_stem = 'http://web.mta.info/developers/'
page = requests.get(subway_turnstile_url_stem + 'turnstile.html')
subway_soup = BeautifulSoup(page.content)

In [3]:
all_links = subway_soup.find_all('a')

In [4]:
dates_links = []
for a_link in all_links:
    try:
        link = a_link.get('href')
        if link[:4] == 'data':
            datestring = link.split('_')[1][:6]
            date = datetime.strptime(datestring, '%y%m%d')
            dates_links.append({'date':date, 'link':subway_turnstile_url_stem+link})
    except:
        print("Error parsing: ",a_link)

Error parsing:  <a name="main-content"> </a>


In [5]:
def get_turnstile_data(list_of_links):
    if isinstance(list_of_links,str):
        list_of_links = [list_of_links]
    df_list = []
    for link in list_of_links:
        week_df = pd.read_csv(link)
        week_df.columns = week_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
        df_list.append(week_df)
    df = pd.concat(df_list)

    return df

In [6]:
relevant_links = [date_link['link'] for date_link in dates_links \
                  if date_link['date'].year >= 2019]

In [7]:
try:
    subway_df = pd.read_pickle('subway_df.pkl')
except:
    print("No pickle of data available, pulling from MTA website ")
    subway_df = get_turnstile_data(relevant_links)
    subway_df = subway_df.loc[(subway_df.desc == "REGULAR")]
    subway_df.to_pickle('subway_df.pkl')

In [8]:
subway_df

,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/15/2022,03:00:00,REGULAR,7679016,2654553
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/15/2022,07:00:00,REGULAR,7679019,2654569
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/15/2022,11:00:00,REGULAR,7679031,2654626
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/15/2022,15:00:00,REGULAR,7679058,2654661
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/15/2022,19:00:00,REGULAR,7679110,2654711
...,...,...,...,...,...,...,...,...,...,...,...
201598,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/04/2019,11:36:26,REGULAR,5554,367
201599,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/04/2019,11:44:04,REGULAR,5554,367
201600,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/04/2019,12:00:00,REGULAR,5554,367
201601,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/04/2019,16:00:00,REGULAR,5554,367


### Data checks and cleaning

As described in the [MTA's field description](http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description.txt), we have several fields in any given data instance. Since entries and exits represent cumulative counter states for a given unit and channel position, I want to calculate the differences between subsequent counter states within a given unit and channel position to get the number of people who passed through that turnstile. We can then aggregate by stations and days to find an approximation of the total number of people who entered or exited a given station.

`C/A      = Control Area (A002)
UNIT     = Remote Unit for a station (R051)
SCP      = Subunit Channel Position represents an specific address for a device (02-00-00)
STATION  = Represents the station name the device is located at
LINENAME = Represents all train lines that can be boarded at this station
           Normally lines are represented by one character.  LINENAME 456NQR repersents train server for 4, 5, 6, N, Q, and R trains.
DIVISION = Represents the Line originally the station belonged to BMT, IRT, or IND   
DATE     = Represents the date (MM-DD-YY)
TIME     = Represents the time (hh:mm:ss) for a scheduled audit event
DESc     = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)
           1. Audits may occur more that 4 hours due to planning, or troubleshooting activities. 
           2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered. 
ENTRIES  = The comulative entry register value for a device
EXITS   = The cumulative exit register value for a device`

I found some of the descriptions from the [Open Data Transit Toolkit](http://transitdatatoolkit.com/lessons/subway-turnstile-data/) and some past analysis on busy commute stations by [the Benson project](https://github.com/Flowinger/MTA-turnstile-data/blob/master/benson_code.ipynb) to be helpful when thinking about this.

In [9]:
(subway_df
 .groupby(['c/a', 'unit', 'scp', 'station', 'date', 'time'])
 .entries.count()
 .reset_index()
 .sort_values("entries", ascending=False)).head()

,c/a,unit,scp,station,date,time,entries
0,A002,R051,02-00-00,59 ST,01/01/2019,03:00:00,1
22058858,R138,R293,00-05-01,34 ST-PENN STA,11/03/2020,21:00:00,1
22058871,R138,R293,00-05-01,34 ST-PENN STA,11/04/2020,21:00:00,1
22058870,R138,R293,00-05-01,34 ST-PENN STA,11/04/2020,17:00:00,1
22058869,R138,R293,00-05-01,34 ST-PENN STA,11/04/2020,13:00:00,1


In [10]:
subway_df.station.unique()

array(['59 ST', '5 AV/59 ST', '57 ST-7 AV', '49 ST', 'TIMES SQ-42 ST',
       '34 ST-HERALD SQ', '28 ST', '23 ST', '14 ST-UNION SQ', '8 ST-NYU',
       'PRINCE ST', 'CANAL ST', 'CITY HALL', 'CORTLANDT ST', 'RECTOR ST',
       'WHITEHALL S-FRY', 'DELANCEY/ESSEX', 'BOWERY', 'CHAMBERS ST',
       'FULTON ST', 'BROAD ST', '7 AV', 'PARK PLACE', 'BOTANIC GARDEN',
       'PROSPECT PARK', 'PARKSIDE AV', 'CHURCH AV', 'BEVERLEY ROAD',
       'CORTELYOU RD', 'NEWKIRK PLAZA', 'AVENUE H', 'AVENUE J',
       'AVENUE M', 'KINGS HWY', 'AVENUE U', 'NECK RD', 'SHEEPSHEAD BAY',
       'BRIGHTON BEACH', 'OCEAN PKWY', 'BOROUGH HALL', 'JAY ST-METROTEC',
       'DEKALB AV', 'ATL AV-BARCLAY', 'UNION ST', '4AV-9 ST',
       'PROSPECT AV', '25 ST', '36 ST', '45 ST', '53 ST', 'BAY RIDGE AV',
       '77 ST', '86 ST', 'BAY RIDGE-95 ST', '8 AV', 'FT HAMILTON PKY',
       'NEW UTRECHT AV', '18 AV', '20 AV', 'BAY PKWY', '9 AV', '50 ST',
       '55 ST', '71 ST', '79 ST', '25 AV', 'BAY 50 ST', 'CONEY IS-STILLW',
      

In [11]:
subway_df['datetime'] = pd.to_datetime(subway_df.date + ' ' + subway_df.time)
subway_df = subway_df.set_index('datetime')
subway_df['activity'] = subway_df['entries'] + subway_df['exits']

I subtract the backward-shifted cumulative count for a given unit from the original cumulative count to get the number of exits or entries within a given 4-hour time interval.

In [12]:
subway_df['entries_per'] = subway_df['entries'] - subway_df.groupby(['c/a', 'unit', 'scp', 'station'])['entries'].shift()
subway_df['exits_per'] = subway_df['exits'] - subway_df.groupby(['c/a', 'unit', 'scp', 'station'])['exits'].shift()
subway_df['activity_per'] = subway_df['activity'] - subway_df.groupby(['c/a', 'unit', 'scp', 'station'])['activity'].shift()

In [13]:
subway_df['entries_per'].describe()

count    3.308310e+07
mean    -3.347390e+02
std      4.815644e+06
min     -2.139015e+09
25%      2.000000e+00
50%      2.600000e+01
75%      1.040000e+02
max      2.139010e+09
Name: entries_per, dtype: float64

There may be times where a counter gets reset, resulting in negative values which I throw out for my analysis.

In [14]:
subway_df = subway_df.fillna(0)
subway_activity_df = subway_df.reset_index()

In [15]:
try:
    subway_activity_df = pd.read_pickle('subway_activity_df.pkl')
except:
    subway_activity_df['week'] = subway_activity_df['datetime'].apply(lambda x: x.isocalendar()[1])
    subway_activity_df['week_date'] = subway_activity_df['datetime'].apply(lambda x: (x - dt.timedelta(days=x.isoweekday() % 7)).date())
    subway_activity_df['year'] = subway_activity_df['datetime'].apply(lambda x: x.year)
    subway_activity_df['date'] = subway_activity_df['datetime'].apply(lambda x: x.date())
    subway_activity_df['year_month_day'] = subway_activity_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

    subway_activity_df = subway_activity_df.loc[subway_activity_df.year >= 2019]
    subway_activity_df.to_pickle('subway_activity_df.pkl')